In [1]:
import random
import networkx as nx
from matplotlib import pyplot as plt
from collections import Counter
import math
from utils.plotTools import plot_qwak
import os
import ast
import numpy as np
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

from scripts import load_list_from_file, write_list_to_file, load_or_generate_data, draw_graph, draw_graph_from_adjacency_matrix,print_matrix
from scripts_tempHelix import generate_static_temporal_helix
from scripts_theoreticalHittingTime import create_transition_matrix,expected_hitting_time

In [13]:
def getWeightedGraph(graph,cw_weight,acw_weight):
    revGraph = graph.reverse()
    for u,v,d in graph.edges(data=True):
        d["weight"] = cw_weight
    for u,v,d in revGraph.edges(data=True):
        d["weight"] = acw_weight
    return nx.compose(graph,revGraph)

def unique_edges_graph(n_nodes):
    """
    Create a graph with `n_nodes` where each node has a unique number of edges.
    
    Parameters:
    - n_nodes: Number of nodes in the graph
    
    Returns:
    - G: A networkx Graph
    """
    if n_nodes <= 1:
        raise ValueError("The number of nodes should be greater than 1.")

    G = nx.Graph()
    total_edges = n_nodes * (n_nodes - 1) // 2  # n(n-1)/2 is the total number of edges for a complete graph of n nodes
    
    if total_edges < sum(range(n_nodes)):  # The total edges of complete graph should be >= sum of first n integers to ensure uniqueness
        raise ValueError(f"Cannot create a graph with {n_nodes} nodes where each has a unique number of edges.")

    # Start building the graph
    current_node = 0
    for i in range(1, n_nodes):
        for j in range(i):
            G.add_edge(current_node, current_node + 1 + j)
        current_node += 1

    return G

In [37]:
def theoretical_hitting_time(P, init, target):
    """
    Calculate the expected hitting time to state z.

    Parameters:
    P (numpy.ndarray): Transition matrix.
    q (numpy.ndarray): Initial state distribution.
    z (int): Target state.

    Returns:
    float: Expected hitting time to state z from state q.
    """
    
    n = P.shape[0]  # Number of states
    # print(n)
    
    if target=='mid':
        target = n//2
        
    if target=='-1':
        target = n-1        
    order = list(range(n))
    order.remove(target)
    order.append(target)
    
    P = P[np.ix_(order, order)]
    # print_matrix(P)
    # print()
        
    P_prime = P[:-1, :-1]

    init_dist = np.zeros(n-1)
    init_dist[init] = 1
    
    target_dist = np.ones(n-1)

    I = np.eye(n-1)

    P_prime_inv = np.linalg.inv(I - P_prime)
        
    hitting_time = np.dot(init_dist, np.dot(P_prime_inv, target_dist))

    return hitting_time

def theoretical_hitting_times_for_staticTempHelix_graphs(reps, init, target):
    hitting_times = []
    for rep in range(1,reps+1):
        G = nx.from_numpy_array(generate_static_temporal_helix(rep))
        P = create_transition_matrix(G)
        print('Temporal Helix')
        print_matrix(P)
        print()
        print_matrix(P.T)
        print()
        hitting_time = theoretical_hitting_time(P, init, target)
        hitting_times.append(hitting_time)
    return hitting_times

def theoretical_hitting_times_for_directedCycle_graphs(n_range, init, target, cw_weight = 1/3, acw_weight = 2/3):
    hitting_times = []
    for n in n_range:
        G = getWeightedGraph(nx.cycle_graph(n,create_using=nx.DiGraph),cw_weight=cw_weight, acw_weight=acw_weight)
        P = create_transition_matrix(G)
        print('Directed Cycle')
        print_matrix(P)
        print()
        print_matrix(P.T)
        print()
        hitting_time = theoretical_hitting_time(P, init, target)
        hitting_times.append(hitting_time)
    return hitting_times

def theoretical_hitting_times_for_irregular_graphs(n_range, init, target):
    hitting_times = []
    for n in n_range:
        G = unique_edges_graph(n)
        print(len(G))
        P = create_transition_matrix(G)
        print('Irregular')
        print_matrix(P)
        print()
        print_matrix(P.T)
        print()
        hitting_time = theoretical_hitting_time(P, init, target)
        hitting_times.append(hitting_time)
    return hitting_times

In [38]:
reps = 1
n_range = range(6,(3*reps+3)+1)
print(list(n_range))

init = 0
target = '-1'

cw_weight=0.6
acw_weight=0.4

theoretical_hitting_times_tempHelix = theoretical_hitting_times_for_staticTempHelix_graphs(reps, init, target)
theoretical_hitting_times_directedCycle = theoretical_hitting_times_for_directedCycle_graphs(n_range, init, target,cw_weight,acw_weight)
theoretical_hitting_times_irregular = theoretical_hitting_times_for_irregular_graphs(n_range, init, target)


[6]
Temporal Helix
|0.50 0.50 0.00 0.00 0.00 0.00 |
|0.17 0.50 0.17 0.17 0.00 0.00 |
|0.00 0.25 0.50 0.00 0.25 0.00 |
|0.00 0.25 0.00 0.50 0.25 0.00 |
|0.00 0.00 0.17 0.17 0.50 0.17 |
|0.00 0.00 0.00 0.00 0.50 0.50 |

|0.50 0.17 0.00 0.00 0.00 0.00 |
|0.50 0.50 0.25 0.25 0.00 0.00 |
|0.00 0.17 0.50 0.00 0.17 0.00 |
|0.00 0.17 0.00 0.50 0.17 0.00 |
|0.00 0.00 0.25 0.25 0.50 0.50 |
|0.00 0.00 0.00 0.00 0.17 0.50 |

Directed Cycle
|0.00 0.60 0.00 0.00 0.00 0.40 |
|0.40 0.00 0.60 0.00 0.00 0.00 |
|0.00 0.40 0.00 0.60 0.00 0.00 |
|0.00 0.00 0.40 0.00 0.60 0.00 |
|0.00 0.00 0.00 0.40 0.00 0.60 |
|0.60 0.00 0.00 0.00 0.40 0.00 |

|0.00 0.40 0.00 0.00 0.00 0.60 |
|0.60 0.00 0.40 0.00 0.00 0.00 |
|0.00 0.60 0.00 0.40 0.00 0.00 |
|0.00 0.00 0.60 0.00 0.40 0.00 |
|0.00 0.00 0.00 0.60 0.00 0.40 |
|0.40 0.00 0.00 0.00 0.60 0.00 |

10
Irregular
|0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 |
|0.33 0.00 0.33 0.33 0.00 0.00 0.00 0.00 0.00 0.00 |
|0.00 0.25 0.00 0.25 0.25 0.25 0.00 0.00 0.00 0.00 